# SQL Avancé

## Révisions

\- Schémas

\- Création de table

## Schémas

Un schéma permet de grouper des tables de même nature pour gérer les permissions plus efficacement.

  

Traditionnellement on va avoir deux schémas différents :

\- data --\> Lecture écriture

\- ref --\> Lecture seule

In [ ]:
IF (EXISTS (
        SELECT [name]
        FROM [master].[sys].[databases]
        WHERE [name] = 'Forum'))
            DROP DATABASE [Forum]

CREATE DATABASE [Forum]

GO

USE [Forum]

In [ ]:
CREATE SCHEMA [data]
GO
CREATE SCHEMA [ref]
GO

# Création de tables

In [ ]:
CREATE TABLE [data].[Utilisateur_UTL] (
    [Id] BIGINT NOT NULL PRIMARY KEY IDENTITY(1, 1), -- Auto incrément
    [Login] NVARCHAR(50) NOT NULL UNIQUE,
    [Password] NVARCHAR(MAX) NOT NULL,
    [CreatedAt] DATETIME NOT NULL DEFAULT GETDATE(),
    [UpdatedAt] DATETIME NOT NULL DEFAULT GETDATE()
)

In [ ]:
CREATE TABLE [ref].[Role_ROL] (
    [Id] SMALLINT NOT NULL PRIMARY KEY IDENTITY(1, 1), -- Id technique pour SGBD
    [Key] VARCHAR(20) NOT NULL UNIQUE, -- Id fonctionnel, "pour l'humain"
    [Description] NVARCHAR(255)
)
CREATE TABLE [ref].[Permission_PER] (
    [Id] SMALLINT NOT NULL PRIMARY KEY IDENTITY(1, 1), -- Id Tech
    [Key] VARCHAR(20) NOT NULL UNIQUE, -- Id fonc
    [Description] NVARCHAR(255)
)

In [ ]:
CREATE TABLE [data].[RoleUtilisateur_RUL] (
    [RolId] SMALLINT NOT NULL,
    [UtlId] BIGINT NOT NULL,
    CONSTRAINT [pk_RUL] PRIMARY KEY ([RolId], [UtlId]),
    CONSTRAINT [fk_RUL_ROL] FOREIGN KEY ([RolId])
        REFERENCES [ref].[Role_ROL]([Id]),
    CONSTRAINT [fk_RUL_UTL] FOREIGN KEY ([UtlId])
        REFERENCES [data].[Utilisateur_UTL]([Id])
)
CREATE TABLE [data].[PermissionRole_PRO] (
    [PerId] SMALLINT NOT NULL,
    [RolId] SMALLINT NOT NULL,
    CONSTRAINT [pk_PRO] PRIMARY KEY ([PerId], [RolId]),
    CONSTRAINT [fk_PRO_PER] FOREIGN KEY ([PerId])
        REFERENCES [ref].[Permission_PER]([Id]),
    CONSTRAINT [fk_PRO_ROL] FOREIGN KEY ([RolId])
        REFERENCES [ref].[Role_ROL]([Id])
)

In [ ]:
CREATE TABLE [data].[Categorie_CAT] (
    [Id] SMALLINT PRIMARY KEY IDENTITY(1, 1) NOT NULL,
    [Nom] NVARCHAR(255) NOT NULL,
    [Cle] NVARCHAR(255) NOT NULL UNIQUE,
    [Ordre] SMALLINT NOT NULL,
    [Creation] DATETIME NOT NULL DEFAULT GETDATE(),
    [Modification] DATETIME NOT NULL DEFAULT GETDATE()
)
CREATE TABLE [data].[Topic_TOP] (
    [Id] INT PRIMARY KEY IDENTITY(1, 1) NOT NULL,
    [CatId] SMALLINT NOT NULL,
    [Nom] NVARCHAR(255) NOT NULL,
    [Cle] NVARCHAR(255) NOT NULL UNIQUE,
    [Ordre] INT NOT NULL,
    [Creation] DATETIME NOT NULL DEFAULT GETDATE(),
    [Modification] DATETIME NOT NULL DEFAULT GETDATE()
)
CREATE TABLE [data].[Sujet_SUJ] (
    [Id] UNIQUEIDENTIFIER PRIMARY KEY NOT NULL DEFAULT NEWID(),
    [TopId] INT NOT NULL,
    [Nom] NVARCHAR(255) NOT NULL,
    [Supprime] BIT NOT NULL DEFAULT 0,
    [Creation] DATETIME NOT NULL DEFAULT GETDATE(),
    [Modification] DATETIME NOT NULL DEFAULT GETDATE()
)
CREATE TABLE [data].[Message_MES] (
    [Id] UNIQUEIDENTIFIER PRIMARY KEY NOT NULL DEFAULT NEWID(),
    [SujId] UNIQUEIDENTIFIER NOT NULL,
    [UtlId] BIGINT NOT NULL,
    [Contenu] NVARCHAR(MAX) NOT NULL,
    [Supprime] BIT NOT NULL DEFAULT 0,
    [Creation] DATETIME NOT NULL DEFAULT GETDATE(),
    [Modification] DATETIME NOT NULL DEFAULT GETDATE()
)

In [ ]:
ALTER TABLE [data].[Topic_TOP]
    ADD CONSTRAINT [fk_top_cat] FOREIGN KEY ([CatId])
        REFERENCES [data].[Categorie_CAT]([Id])

ALTER TABLE [data].[Sujet_SUJ]
    ADD CONSTRAINT [fk_suj_top] FOREIGN KEY ([TopId])
        REFERENCES [data].[Topic_TOP]([Id])

ALTER TABLE [data].[Message_MES]
    ADD CONSTRAINT [fk_mes_suj] FOREIGN KEY ([SujId])
        REFERENCES [data].[Sujet_SUJ]([Id]),
    CONSTRAINT [fk_mes_utl] FOREIGN KEY ([UtlId])
        REFERENCES [data].[Utilisateur_UTL]([Id])
